In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

# Step 1: 데이터 전처리 - AutoTokenizer 사용
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 데이터셋 로드
dataset = load_dataset("csv", data_files="./datas/justice/justice_train.csv")

# 텍스트와 레이블 추출
texts = dataset['train']['scenario']
labels = dataset['train']["label"]

# Step 2: 레이블링 - 정의로우면 0, 아니면 1로 레이블링
labels = torch.tensor(labels)
labels = 1 - labels  # 0과 1을 반대로 매핑

# Step 3: 데이터셋 분할 - 트레인 데이터셋과 테스트 데이터셋
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 토큰화 및 인코딩
train_encoded_inputs = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
test_encoded_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

# Step 4: 모델 학습 - Hugging Face Hub에서 모델 가져오기
model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten_tomatoes")

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-1,
    per_device_train_batch_size=512,  # 배치 크기를 32로 조정
    per_device_eval_batch_size=512,  # 배치 크기를 32로 조정
    num_train_epochs=1,  # 학습 에폭 수를 1로 줄임
    weight_decay=0.1,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Dataset.from_dict({
        'input_ids': train_encoded_inputs['input_ids'],
        'attention_mask': train_encoded_inputs['attention_mask'],
        'labels': train_labels,
    }),
    eval_dataset=Dataset.from_dict({
        'input_ids': test_encoded_inputs['input_ids'],
        'attention_mask': test_encoded_inputs['attention_mask'],
        'labels': test_labels,
    }),
)

# 모델 학습
trainer.train()

# Step 5: 최종 결과 측정 - 정확도로 측정
eval_result = trainer.evaluate(eval_dataset=torch.utils.data.TensorDataset(test_encoded_inputs['input_ids'], test_encoded_inputs['attention_mask'], test_labels))
accuracy = eval_result['eval_accuracy']
print("Accuracy:", accuracy)

Found cached dataset csv (C:/Users/User/.cache/huggingface/datasets/csv/default-6304ef94639b87b5/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\User\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17432
  Num Epochs = 1
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 35
  Number of trainable parameters = 109483778


Epoch,Training Loss,Validation Loss


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

# 1. 데이터 전처리
class EthicsDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self.data = pd.read_csv(data_path)
        self.labels = self.data['label'].values
        self.texts = self.data['scenario'].values
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label)
        }

# 2. 데이터셋 로드 및 전처리
data_path = '/Users/minsoo/Downloads/ethics/justice/justice_train.csv'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
dataset = EthicsDataset(data_path, tokenizer)
train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# 3. 모델 학습
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

model.to(device)

for epoch in range(3):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_loader)
    
    print(f"Epoch {epoch+1}:")
    print(f"  Training Loss: {average_loss}")
    
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in eval_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            
            total += labels.size(0)
            correct += (predictions == labels).sum().item()
    
    accuracy = correct / total
    print(f"  Evaluation Accuracy: {accuracy}")

# 4. 최종 결과
print(f"Final Evaluation Accuracy: {accuracy}")

ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tf_model.h5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2d96c4790>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
from transformers import AutoTokenizer

# 말뭉치 데이터셋 준비
texts = [...]  # 말뭉치 데이터셋 (입력 텍스트)
labels = [...]  # 말뭉치에 대한 레이블 (출력 예측값)

# AutoTokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 텍스트 데이터 전처리 및 토큰화
input_data = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# 레이블 데이터 전처리
label_data = tf.keras.utils.to_categorical(labels)

# 모델 구성, 컴파일, 훈련은 이전 예시 코드와 동일하게 진행하면 됩니다.
# ...

# 예측하기
new_texts = [...]  # 예측할 텍스트 데이터

# 텍스트 데이터 전처리 및 토큰화
new_data = tokenizer(new_texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# 예측 수행
predictions = model.predict(new_data["input_ids"])

# 위의 코드에서는 다음과 같은 단계를 거칩니다:

# 말뭉치 데이터셋을 준비합니다. texts는 입력 텍스트 데이터, labels는 해당 텍스트에 대한 레이블입니다.
# AutoTokenizer를 사용하여 원하는 토크나이저를 자동으로 로드합니다. 위 예시에서는 "bert-base-uncased" 모델의 토크나이저를 사용하도록 설정하였습니다.
# 텍스트 데이터를 전처리하고 토큰화합니다. tokenizer 객체를 사용하여 텍스트를 토큰화하고, 패딩과 자르기(truncation)를 적용하여 일관된 길이로 맞춥니다. return_tensors 매개변수를 통해 TensorFlow에서 사용할 수 있는 텐서 형태로 출력합니다.
# 레이블 데이터를 전처리합니다. 레이블을 범주형 형식으로 변환합니다.
# 모델 구성, 컴파일, 훈련은 이전 예시 코드와 동일하게 진행하면 됩니다.
# 예측을 수행합니다. 새로운 텍스트 데이터를 전처리하고 토큰화한 후, 학습된 모델을 사용하여 예측값을 얻습니다.
# AutoTokenizer를 사용하면 원하는 언어 모델의 토크나이저를 손쉽게 선택할 수 있으며, 다양한 모델에 대한 호환성을 제공합니다. 위의 코드를 사용하면 자동으로 선택된 토크나이저를 활용하여 텍스트 데이터를 처리할 수 있습니다.






